# XGBoost and GA with GLPK
This notebook contains the implementation of a Genetic Algorithm to carry out feature selection. This hasn't produced a reasonable result however (in fact, it can't find a tree), as it appears to have selected too many features for use with BNP-OCT (35 features currently).

In [1]:
from numpy import loadtxt
import pandas as pd
from numpy import sort
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

from lsopt.tree import OptimalTreeClassifier ## M-OCT propsed by Liu & Allen
from lsopt.tree import BinNodePenaltyOptimalTreeClassifier ## BNP-OCT propsed by Liu & Allen
# from lsopt.tree import OldOptimalTreeClassifier ## OCT proposed by Bertsimas & Dunn

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import tree 

import graphviz

%load_ext autoreload
%autoreload 2

In [2]:
data_path_fraud = './data/fraud_data_m_oct_ready.csv'
fraud_data = pd.read_csv(data_path_fraud)
fraud_data

,phy_same,phy_count,period,periodadmt,age,alife,provider_InscClaimAmtReimbursed_mean,provider_DeductibleAmtPaid_mean,provider_NoOfMonths_PartACov_mean,provider_NoOfMonths_PartBCov_mean,...,diag1_OPAnnualDeductibleAmt_mean,diag1_InscClaimAmtReimbursed_std,diag1_DeductibleAmtPaid_std,diag1_NoOfMonths_PartACov_std,diag1_NoOfMonths_PartBCov_std,diag1_IPAnnualReimbursementAmt_std,diag1_IPAnnualDeductibleAmt_std,diag1_OPAnnualReimbursementAmt_std,diag1_OPAnnualDeductibleAmt_std,PotentialFraud
0,2.960000,1.600000,1.440000,1.000000,80.240000,1.000000,4185.600000,213.600000,12.000000,12.000000,...,543.045084,3482.066310,161.353027,0.569945,0.424192,12941.552350,1205.297144,2450.076771,661.506672,Not-Fraud
1,2.439394,1.530303,3.674242,2.424242,71.371212,0.992424,4588.409091,502.166667,11.818182,11.871212,...,676.313985,4017.871066,260.257069,0.726572,0.653285,12620.604410,1226.306633,3369.338617,848.213675,Fraud
2,2.818792,1.604027,1.429530,0.000000,73.516779,0.993289,350.134228,2.080537,11.865772,11.959732,...,694.246881,1536.290845,113.086257,0.667719,0.577420,11016.516940,1111.592405,2972.377916,808.138208,Not-Fraud
3,2.731330,1.599142,1.088412,0.000000,71.783691,0.996567,241.124463,3.175966,11.907296,11.939914,...,630.805985,1234.005090,91.141252,0.657071,0.565930,10021.329570,957.701391,2727.944083,737.419878,Fraud
4,2.736111,1.527778,0.958333,0.222222,70.583333,0.986111,468.194444,45.333333,11.833333,11.833333,...,606.550334,1519.425993,103.302166,0.626542,0.520122,10565.761430,1126.358206,2486.827069,682.279276,Not-Fraud
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,2.548387,1.548387,2.806452,0.000000,75.677419,1.000000,127.741935,1.612903,12.000000,12.000000,...,539.926413,1278.578369,105.316369,0.642724,0.506470,10069.067870,1048.496358,2331.087492,676.226785,Not-Fraud
1005,2.500000,1.500000,0.100000,0.000000,78.000000,1.000000,72.000000,0.000000,12.000000,12.000000,...,544.784235,589.615472,61.510304,0.690826,0.473344,10885.075840,1026.840019,2547.341333,723.822292,Not-Fraud
1006,3.000000,2.000000,0.000000,0.000000,74.000000,1.000000,50.000000,0.000000,12.000000,12.000000,...,613.157895,4778.012673,463.684066,0.600751,0.710275,13241.321690,1469.095843,3203.267596,911.406530,Not-Fraud
1007,2.333333,1.333333,2.266667,0.000000,73.933333,1.000000,296.000000,0.000000,12.000000,12.000000,...,566.953462,2506.463260,131.832995,0.726815,0.354062,11262.841610,1196.045563,2691.729344,736.415563,Not-Fraud


In [3]:
X = fraud_data.iloc[:, 0:47].to_numpy()

In [4]:
y = fraud_data["PotentialFraud"].apply(lambda val: 0 if val == "Not-Fraud" else 1).to_numpy()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]    
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))  

Accuracy: 93.56%


In [6]:
thresholds = sort(model.feature_importances_)
max_acc = -1
true_thresh = -1
for thresh in thresholds:
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    
    selection_model = XGBClassifier()
    selection_model.fit(select_X_train, y_train)
    
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    
    predictions = [round(value) for value in y_pred]
    
    accuracy = accuracy_score(y_test, predictions)
    if accuracy >= max_acc and select_X_train.shape[1] < 20:
        max_acc = accuracy
        true_thresh = thresh
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

Thresh=0.000, n=47, Accuracy: 93.56%
Thresh=0.000, n=47, Accuracy: 93.56%
Thresh=0.000, n=47, Accuracy: 93.56%
Thresh=0.000, n=47, Accuracy: 93.56%
Thresh=0.000, n=47, Accuracy: 93.56%
Thresh=0.000, n=47, Accuracy: 93.56%
Thresh=0.005, n=41, Accuracy: 93.56%
Thresh=0.009, n=40, Accuracy: 94.06%
Thresh=0.009, n=39, Accuracy: 94.06%
Thresh=0.009, n=38, Accuracy: 94.06%
Thresh=0.010, n=37, Accuracy: 94.06%
Thresh=0.011, n=36, Accuracy: 94.55%
Thresh=0.011, n=35, Accuracy: 94.06%
Thresh=0.011, n=34, Accuracy: 94.06%
Thresh=0.012, n=33, Accuracy: 95.05%
Thresh=0.012, n=32, Accuracy: 94.06%
Thresh=0.012, n=31, Accuracy: 95.54%
Thresh=0.012, n=30, Accuracy: 94.06%
Thresh=0.013, n=29, Accuracy: 95.05%
Thresh=0.013, n=28, Accuracy: 93.56%
Thresh=0.014, n=27, Accuracy: 94.06%
Thresh=0.014, n=26, Accuracy: 94.06%
Thresh=0.014, n=25, Accuracy: 94.06%
Thresh=0.015, n=24, Accuracy: 95.54%
Thresh=0.015, n=23, Accuracy: 94.55%
Thresh=0.016, n=22, Accuracy: 93.56%
Thresh=0.017, n=21, Accuracy: 94.06%
T

In [7]:
print(true_thresh)

0.032147862


In [8]:
from lsopt.tree import OptimalTreeClassifier ## M-OCT propsed by Liu & Allen
from lsopt.tree import BinNodePenaltyOptimalTreeClassifier ## BNP-OCT propsed by Liu & Allen
selection = SelectFromModel(model, threshold=true_thresh, prefit=True)
select_X_train = selection.transform(X_train)

# OCT parameters
max_depth = 3
min_samples_leaf = 10
alpha = 0.00005
time_limit = 15 # minute
mip_gap_tol = 0.01  # optimal gap percentage
mip_focus = 'balance'
mip_polish_time = None
warm_start = False
log_file = None
fp_heur = True
backtrack = "bestb"

# Construct BNP-OCT classifier
selection_model = BinNodePenaltyOptimalTreeClassifier(max_depth=max_depth,min_samples_leaf=min_samples_leaf,
                                  alpha=alpha,
                                  criterion="gini",
                                  solver="gurobi",
                                  time_limit=time_limit,
                                  verbose=True,
                                  warm_start=warm_start,
                                  log_file=log_file,
                                  solver_options={'mip_cuts': 'auto',
                                                  'mip_gap_tol': mip_gap_tol,
                                                  'mip_focus': mip_focus,
                                                  'mip_polish_time': mip_polish_time,
                                                  }
                                  )

names = []
for i in range(len(model.feature_importances_)):
    if model.feature_importances_[i] >= true_thresh:
        names.append(fraud_data.iloc[:, [i]].columns[0])
print(len(names))

7


In [9]:
print(names)

['periodadmt', 'alife', 'provider_NoOfMonths_PartACov_mean', 'provider_InscClaimAmtReimbursed_std', 'provider_NoOfMonths_PartBCov_std', 'provider_IPAnnualDeductibleAmt_std', 'provider_OPAnnualDeductibleAmt_std']


# Explanation of below code
Alright, there's a few functions here, so lets go through them one by one.

### Split
This one is pretty straitforward, its the one that actually splits the data into a training set and a testing set

### acc_score
Returns the accuracy of each model

### plot
For plotting any results of the model, if I need to visualize it

### initialization_of_population
To initialize a random population.

### fitness_score
Returns the best parents for the next run, along with their fitness scores

### selection
Selects the best parents at the end of every run

### crossover
Picks half of the first parents, and half of the second

### mutation
Randomly flip the a feature from True to False (holding true to the principle of the genetic algorithm)

### generations
Executes all the above functions for the specified number of generations

In [10]:
def split(df,label):
    selection = SelectFromModel(model, threshold=true_thresh, prefit=True)
    X_tr = select_X_train
    X_te = X_test
    Y_tr = y_train
    Y_te = y_test
    return X_tr, X_te, Y_tr, Y_te

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score
from xgboost import XGBClassifier
import random

classifiers = ['LinearSVM', 'RadialSVM', 
               'Logistic',  'RandomForest', 
               'AdaBoost',  'DecisionTree', 
               'KNeighbors','GradientBoosting', 'BNP-OCT']

models = [svm.SVC(kernel='linear'),
          svm.SVC(kernel='rbf'),
          LogisticRegression(max_iter = 1000),
          RandomForestClassifier(n_estimators=200, random_state=0),
          AdaBoostClassifier(random_state = 0),
          DecisionTreeClassifier(random_state=0),
          KNeighborsClassifier(),
          XGBClassifier(random_state=0),
          BinNodePenaltyOptimalTreeClassifier(max_depth=max_depth,min_samples_leaf=min_samples_leaf,
                                  alpha=alpha,
                                  criterion="gini",
                                  solver="gurobi",
                                  time_limit=time_limit,
                                  verbose=True,
                                  warm_start=warm_start,
                                  log_file=log_file,
                                  solver_options={'mip_cuts': None,
                                                  'mip_gap_tol': mip_gap_tol,
                                                  'mip_focus': mip_focus,
                                                  'mip_polish_time': mip_polish_time,
                                                  }
                                  )]


def acc_score(df,label):
    Score = pd.DataFrame({"Classifier":classifiers})
    j = 0
    acc = []
    X_train = select_X_train
    X_test = select_X_train
    Y_train = y_train
    Y_test = y_train
    for i in models:
        model = i
        model.fit(X_train,Y_train)
        predictions = model.predict(X_test)
        acc.append(accuracy_score(Y_test,predictions))
        j = j+1     
    Score["Accuracy"] = acc
    Score.sort_values(by="Accuracy", ascending=False,inplace = True)
    Score.reset_index(drop=True, inplace=True)
    return Score

def plot(score,x,y,c = "b"):
    gen = [1,2,3,4,5]
    plt.figure(figsize=(6,4))
    ax = sns.pointplot(x=gen, y=score,color = c )
    ax.set(xlabel="Generation", ylabel="Accuracy")
    ax.set(ylim=(x,y))

# Initialization of population
def initilization_of_population(size):
    population = [{"min_samples_leaf": 10, "mip_gap_tol": 0.01, "alpha": 0.00005, "time_limit": 10}, 
                  {"min_samples_leaf": 7, "mip_gap_tol": 0.05, "alpha": 0.00003, "time_limit": 15}, 
                  {"min_samples_leaf": 5, "mip_gap_tol": 0.04, "alpha": 0.0001, "time_limit": 21}, 
                  {"min_samples_leaf": 9, "mip_gap_tol": 0.1, "alpha": 0.00008, "time_limit": 27}]
    return population


def fitness_score(population):
    scores = []
    for chromosome in population:
        model = BinNodePenaltyOptimalTreeClassifier(max_depth=max_depth,min_samples_leaf=chromosome["min_samples_leaf"],
                                  alpha=chromosome["alpha"],
                                  criterion="gini",
                                  solver="gurobi",
                                  time_limit=chromosome["time_limit"],
                                  verbose=True,
                                  warm_start=warm_start,
                                  log_file=log_file,
                                  solver_options={'mip_cuts': 'auto',
                                                  'mip_gap_tol': chromosome["mip_gap_tol"],
                                                  'mip_focus': mip_focus,
                                                  'mip_polish_time': mip_polish_time,
                                                  }
                                  )
        model.fit(select_X_train, y_train)
        predictions = model.predict(select_X_train)
        scores.append(accuracy_score(y_train, predictions))
    scores, population = np.array(scores), np.array(population)
    inds = np.argsort(scores)
    return list(scores[inds][::-1]), list(population[inds][::-1])
        
    # scores = []
    # for chromosome in population:
    #     logmodel.fit(select_X_train[:, chromosome],y_train)         
    #     predictions = logmodel.predict(select_X_train[:, chromosome])
    #     scores.append(accuracy_score(y_train,predictions))
    # scores, population = np.array(scores), np.array(population) 
    # inds = np.argsort(scores)                                    
    # return list(scores[inds][::-1]), list(population[inds,:][::-1]) 

def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen


def crossover(pop_after_sel):
    pop_nextgen = pop_after_sel
    for i in range(0,len(pop_after_sel),2):
        new_par = {}
        child_1, child_2 = pop_nextgen[i] , pop_nextgen[i+1]
        num = random.uniform(0, 1)
        if (num > 0.5):
            new_par["min_samples_leaf"] = child_1["min_samples_leaf"]
            new_par["mip_gap_tol"] = child_1["mip_gap_tol"]
            new_par["alpha"] = child_2["alpha"]
            new_par["time_limit"] = child_2["time_limit"]
        else:
            new_par["min_samples_leaf"] = child_2["min_samples_leaf"]
            new_par["mip_gap_tol"] = child_2["mip_gap_tol"]
            new_par["alpha"] = child_1["alpha"]
            new_par["time_limit"] = child_1["time_limit"]
        pop_nextgen.append(new_par)
    return pop_nextgen
    # pop_nextgen = pop_after_sel
    # for i in range(0,len(pop_after_sel),2):
    #     new_par = []
    #     child_1, child_2 = pop_nextgen[i] , pop_nextgen[i+1]
    #     new_par = np.concatenate((child_1[:len(child_1)//2],child_2[len(child_1)//2:]))
    #     pop_nextgen.append(new_par)
    # return pop_nextgen

def mutation(pop_after_cross,mutation_rate):   
    mutation_range = int(mutation_rate*3)
    pop_next_gen = []
    for n in range(len(pop_after_cross)):
        chromo = pop_after_cross[n]
        # rand_num = random.randint(0, 2)
        # if rand_num == 0:
        #     chromo
        pop_next_gen.append(chromo)
    return pop_next_gen

def generations(df,label,size,n_feat,n_parents,mutation_rate,n_gen,X_train,
                                   X_test, Y_train, Y_test):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print("pop_after_fit", pop_after_fit)
        print('Best score in generation',i+1,':',scores[:1])  #2
        pop_after_sel = selection(pop_after_fit,n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross,mutation_rate)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo,best_score

# Explanation of cell below
The below cell essentially generates the accuracy of all the possible model classifiers that can be used (it appears that XGBoost is best, along with AdaBoost). Basically, what we're doing is running each model on the 7 features pre-selected with XGBoost, to determine which one runs in the most accurarte fashion.

In [11]:
score = acc_score(select_X_train, y_train)
score

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2433923
WLS license 2433923 - registered to Sagata UAE
Read LP format model from file /var/folders/kb/sxk47lqj0xd8j0x92mykb0rr0000gn/T/tmpgbb5vfcg.pyomo.lp
Reading time = 0.17 seconds
x1: 26210 rows, 6622 columns, 438053 nonzeros
Set parameter TimeLimit to value 900
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

WLS license 2433923 - registered to Sagata UAE
Optimize a model with 26210 rows, 6622 columns and 438053 nonzeros
Model fingerprint: 0x9ef9563b
Variable types: 6489 continuous, 133 integer (133 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [5e-05, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 2026 rows and 451 columns
Presolve time: 1

,Classifier,Accuracy
0,RandomForest,1.000000
1,DecisionTree,1.000000
2,GradientBoosting,0.998761
3,AdaBoost,0.944238
4,BNP-OCT,0.921933
5,KNeighbors,0.909542
6,RadialSVM,0.895911
7,Logistic,0.887237
8,LinearSVM,0.881041


## Explanation Of Below Cell

Below, I'm running the GA with BNP-OCT as its base. Basically, using the 7 features already selected by XGBoost, the idea is the GA is being used for further feature selection here. Basically, what its doing is running over multiple possible feature combinations with BNP-OCT as the classifier that its exploring. The algorithm is selecting the most accurate BNP-OCT outputs with that, and continuing to recombine, mutate etc. Currently, its taking very long, because each generation has about 2 possible solutions being explored, and those are recombined for the next set, with each possible model taking 10 minutes to train, so all combination can take upto and 1 to an hour and half to run. Also, there's a posssibility that some solutions are invalid, so if we hit upon those in the GA, it means it came across an invalid solution. Currently, I'm fixing that by removing the mutation rate, and also trying to increase the set of explored parent solution beforem moving generations (as the greater the number of parent solutions, the greater chance that the GA will ignore some in the next generation, only selecting the correct ones, but its a balancing act of not having too many parents that training takes an absurd amount of time. With XGBoost, the number of required parents was 64, which is too large a number of BNP-OCT to explore with GA).

## Next steps
If this ends up not producing anything reasonable, I could try exploring GA on hyperparameters (but that would be a lot longer on implementation, as the GA would have to be altered in a fundamental way). 

Finally, we should check with Ted, if he wants GA to be used as an alternative to MILP, as that would probably be the multi-month project of delving into _base.py and tree.py and altering those algos.

In [12]:
# Run the GA
# logmodel = BinNodePenaltyOptimalTreeClassifier(max_depth=max_depth,min_samples_leaf=min_samples_leaf,
#                                   alpha=alpha,
#                                   criterion="gini",
#                                   solver="gurobi",
#                                   time_limit=time_limit,
#                                   verbose=True,
#                                   warm_start=warm_start,
#                                   log_file=log_file,
#                                   solver_options={'mip_cuts': None,
#                                                   'mip_gap_tol': mip_gap_tol,
#                                                   'mip_focus': mip_focus,
#                                                   'mip_polish_time': mip_polish_time,
#                                                   }
#                                   )
# X_train,X_test, Y_train, Y_test = split(X, y)
chromo_df_bc,score_bc=generations(select_X_train,y_train,size=4,n_feat=select_X_train.shape[1],n_parents=2,mutation_rate=0.00,n_gen=2,
                         X_train = select_X_train,X_test = select_X_train,Y_train = y_train,Y_test = y_train)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2433923
WLS license 2433923 - registered to Sagata UAE
Read LP format model from file /var/folders/kb/sxk47lqj0xd8j0x92mykb0rr0000gn/T/tmpws8koinz.pyomo.lp
Reading time = 0.30 seconds
x1: 26210 rows, 6622 columns, 438053 nonzeros
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

WLS license 2433923 - registered to Sagata UAE
Optimize a model with 26210 rows, 6622 columns and 438053 nonzeros
Model fingerprint: 0x9ef9563b
Variable types: 6489 continuous, 133 integer (133 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [5e-05, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 2026 rows and 451 columns
Presolve time: 2

In [13]:
# Select the features indicated by the GA
print(chromo_df_bc)
print(score_bc)
# names = []
# select_X_train = []
# for i in range(len(chromo_df_bc[2])):
#     if chromo_df_bc[2][i] == True:
#         names.append(fraud_data.iloc[:, [i]].columns[0])
#         select_X_train.append(X_train[:, i])
# select_X_train = np.asarray(select_X_train).T
# print(len(names))
# print(select_X_train.shape)
# print(X_train.shape)
# print(Y_train.shape)
        

[{'min_samples_leaf': 9, 'mip_gap_tol': 0.1, 'alpha': 8e-05, 'time_limit': 27}, {'min_samples_leaf': 9, 'mip_gap_tol': 0.1, 'alpha': 0.0001, 'time_limit': 21}]
[0.9219330855018587, 0.9219330855018587]


In [14]:
names_1 = []
select_X_train_1 = []
for i in range(len(chromo_df_bc[0])):
    if chromo_df_bc[0][i] == True:
        names_1.append(names[i])
        select_X_train_1.append(select_X_train[:, i])
select_X_train_1 = np.asarray(select_X_train_1).T
print(select_X_train_1.shape)
print(select_X_train.shape)

KeyError: 0

In [15]:
from lsopt.tree import OptimalTreeClassifier ## M-OCT propsed by Liu & Allen
from lsopt.tree import BinNodePenaltyOptimalTreeClassifier ## BNP-OCT propsed by Liu & Allen
# selection = SelectFromModel(model, threshold=true_thresh, prefit=True)
# select_X_train = selection.transform(X_train)

# OCT parameters
max_depth = 3
min_samples_leaf = chromo_df_bc[0]["min_samples_leaf"]
alpha = chromo_df_bc[0]["alpha"]
time_limit = chromo_df_bc[0]["time_limit"]  # minute
mip_gap_tol = chromo_df_bc[0]["mip_gap_tol"]  # optimal gap percentage
mip_focus = 'balance'
mip_polish_time = None
# fp_heur = True
# backtrack = "bestb"

warm_start = False
log_file = None



# Construct BNP-OCT classifier
selection_model = BinNodePenaltyOptimalTreeClassifier(max_depth=max_depth,min_samples_leaf=min_samples_leaf,
                                  alpha=alpha,
                                  criterion="gini",
                                  solver="gurobi",
                                  time_limit=time_limit,
                                  verbose=True,
                                  warm_start=warm_start,
                                  log_file=log_file,
                                  solver_options={'mip_cuts': None,
                                                  'mip_gap_tol': mip_gap_tol,
                                                  'mip_focus': mip_focus,
                                                  'mip_polish_time': mip_polish_time,
                                                  }
                                  )

In [16]:
selection_model.fit(select_X_train, y_train)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2433923
WLS license 2433923 - registered to Sagata UAE
Read LP format model from file /var/folders/kb/sxk47lqj0xd8j0x92mykb0rr0000gn/T/tmpwddldklm.pyomo.lp
Reading time = 0.18 seconds
x1: 26210 rows, 6622 columns, 438053 nonzeros
Set parameter TimeLimit to value 1620
Set parameter MIPGap to value 0.1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

WLS license 2433923 - registered to Sagata UAE
Optimize a model with 26210 rows, 6622 columns and 438053 nonzeros
Model fingerprint: 0xbedeb36a
Variable types: 6489 continuous, 133 integer (133 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e-05, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 2026 rows and 451 columns
Presolve time: 1

In [17]:
# Make prediction
# selection = SelectFromModel(model, threshold=true_thresh, prefit=True)
# select_X_test = selection.transform(X_test)
y_pred = selection_model.predict(X=select_X_train)
y_pred_prob = selection_model.predict_proba(X=select_X_train)

# Check confusion matrix
print("Confusion Matrix :")
print(confusion_matrix(y_true=y_train,
                       y_pred=y_pred))

print(classification_report(y_true=y_train,
                            y_pred=y_pred))

# Plot Optimal Tree
feature_names = names
class_names = ['Not-Fraud', 'Fraud']

dot_data = tree.export_graphviz(selection_model,
                                out_file=None,
                                feature_names=feature_names,
                                class_names=class_names,
                                label='all',
                                impurity=True,
                                node_ids=True,
                                filled=True,
                                rounded=True,
                                leaves_parallel=True,
                                special_characters=False)

graph = graphviz.Source(dot_data)
graph.format = 'png'
graph.render(filename='optimal_tree_fraud', directory='', view=True)

Confusion Matrix :
[[707  11]
 [ 52  37]]
              precision    recall  f1-score   support

           0       0.93      0.98      0.96       718
           1       0.77      0.42      0.54        89

    accuracy                           0.92       807
   macro avg       0.85      0.70      0.75       807
weighted avg       0.91      0.92      0.91       807



'optimal_tree_fraud.png'

In [18]:
print(dot_data)

digraph Tree {
node [shape=box, style="filled, rounded", color="black", fontname="helvetica"] ;
graph [ranksep=equally, splines=polyline] ;
edge [fontname="helvetica"] ;
0 [label="node #0\nperiodadmt <= 0.415\ngini = 0.196\nsamples = 807\nvalue = [718, 89]\nclass = Not-Fraud", fillcolor="#e89152"] ;
1 [label="node #1\ngini = 0.069\nsamples = 529\nvalue = [510, 19]\nclass = Not-Fraud", fillcolor="#e68640"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="node #2\nprovider_NoOfMonths_PartBCov_std <= 0.413\ngini = 0.377\nsamples = 278\nvalue = [208, 70]\nclass = Not-Fraud", fillcolor="#eeab7c"] ;
0 -> 2 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;
3 [label="node #3\ngini = 0.251\nsamples = 197\nvalue = [168, 29]\nclass = Not-Fraud", fillcolor="#e9975b"] ;
2 -> 3 ;
4 [label="node #4\nprovider_NoOfMonths_PartBCov_std <= 1.248\ngini = 0.5\nsamples = 81\nvalue = [40, 41]\nclass = Fraud", fillcolor="#fafdfe"] ;
2 -> 4 ;
5 [label="node #5\ngini = 0.353\nsamp

In [ ]:
count = 0
for element in y_pred:
    if element == 1:
        count += 1
print(count)

0


In [ ]:
print(selection_model)

In [ ]:
Image('optimal_tree_fraud.png')

NameError: name 'Image' is not defined

In [ ]:
import pickle
with open('model_pkl', 'wb') as file:
    pickle.dump(selection_model, file)


AttributeError: Can't pickle local object 'solve_oct_MILP_BIN.<locals>.obj_rule'